This notebook is intended to aid the construction of a first pointing model with the Auxiliary Telescope. 

At this early stages, and given the small field-of-view of the telescope, it is really hard to get a start on the field. To help on the process, this notebook implements a Gridding routine. The routine will scan around the current position of the telescope and waits for the user to specify if is should continue, go to the next or previous pointing or stop. 



In [1]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

## Disable auto download of iers data. 

The main reason for disabling this is that the containers on the control network do not have access to the internet. Trying to download data when there is no outside access means a long time before the connection times out. 

In [2]:
from astropy.utils import iers
iers.conf.auto_download = False

## Check value of the `LSST_DDS_DOMAIN` variable.

For the AT early work at the summit, the expected value for `LSST_DDS_DOMAIN=lsatmcs`

In [3]:
import os
print(os.environ["LSST_DDS_DOMAIN"])

lsatmcs


## Create a domain and remotes

The ATTCS class uses the following remotes:

  - atmcs 
  - atptg
  - ataos
  - atpneumatics 
  - athexapod
  - atdome
  - atdometrajectory


In [4]:
d = salobj.Domain()

In case, you need to build the idl files, copy the following command to a cell and run it. 

```
%%script bash 
make_idl_files.py ATMCS ATPtg ATAOS ATPneumatics ATHexapod ATDome ATDomeTrajectory
```

In [ ]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")

In [ ]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task)

In [5]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [6]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

[None, None, None, None, None, None, None]

In [248]:
await salobj.set_summary_state(ataos, salobj.State.STANDBY, settingsToApply='current')

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 34 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 33 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 34 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 34 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 34 messages


[<State.DISABLED: 1>, <State.STANDBY: 5>]

In [14]:
await ataos.cmd_enableCorrection.set_start(m1=True)

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 37 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 36 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 37 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 37 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 37 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 37 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 37 messages
RemoteEvent(ATPtg, 0, mountDataWarning) falling behind; read 32 

RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 37 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 37 messages


In [ ]:
await ataos.cmd_disableCorrection.set_start(m1=True)

In [234]:
await atmcs.cmd_standby.start()

RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 28 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 28 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 28 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 27 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 27 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 28 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 28 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 28 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 28 messages
RemoteTelemetry(ATPtg, 0

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1487062206, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='ERROR: Command standby rejected while in FaultState state.tate state.')

In [247]:
await salobj.set_summary_state(atmcs, salobj.State.STANDBY, timeout=120)
# await atmcs.cmd_standby.start(timeout=60)

RemoteTelemetry(ATDome, 1, position) falling behind; read 36 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 36 messages


[<State.DISABLED: 1>, <State.STANDBY: 5>]

In [249]:
# await atpne.cmd_standby.start(timeout=30)
await salobj.set_summary_state(atpne, salobj.State.STANDBY, timeout=120)

RemoteEvent(ATDome, 1, heartbeat) falling behind; read 13 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 13 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 13 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 13 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 12 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 13 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 13 messages


[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [45]:
await atpne.cmd_openMasterAirSupply.start()

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 100 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling b

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1661760039, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='ERROR: Command masterOpenValve rejected while in EnabledState state.')

RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 100 messages


In [46]:
await atpne.cmd_openInstrumentAirValve.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1361087480, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='ERROR: Command instOpenValve rejected while in EnabledState state.')

In [43]:
await atpne.cmd_m1OpenAirValve.start()

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 37 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 38 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 38 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 38 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 38 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99

In [44]:
await atpne.cmd_m1SetPressure.set_start(pressure=100000)

RemoteEvent(ATMCS, 0, target) falling behind; read 98 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 98 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 24 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 25 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 25 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 25 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 25 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 97 elements
falling behind; queue contains 96 elements
falling behind; queue contains 95 elements


In [250]:
await atdome.evt_heartbeat.next(flush=True, timeout=30)
await salobj.set_summary_state(atdome, salobj.State.STANDBY, timeout=120, settingsToApply='test')

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 54 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 54 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 11 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 11 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 11 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 10 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 10 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 54 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 54 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 54 messages


[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [242]:
await atdomtraj.evt_heartbeat.next(flush=True, timeout=10)
await salobj.set_summary_state(atdomtraj, salobj.State.STANDBY, timeout=120)

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 23 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMoto

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [251]:
await salobj.set_summary_state(atptg, salobj.State.STANDBY, timeout=120)

RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 22 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages


[<State.FAULT: 3>, <State.STANDBY: 5>]

In [241]:
await atptg.cmd_azElTarget.set_start(azDegs=0., elDegs=80.)

In [ ]:
await atptg.cmd_stopTracking.start()

In [ ]:
await atmcs.cmd_stopTracking.start()

In [ ]:
ss = await atptg.evt_summaryState.next(flush=False, timeout=10)

In [ ]:
print(salobj.State(ss.summaryState))

In [ ]:
await atdome.cmd_moveAzimuth.set_start(azimuth=270.)

In [ ]:
await atdome.cmd_stopMotion.start()

In [ ]:
await atdome.cmd_homeAzimuth.start()

In [ ]:
await atdome.cmd_openShutter.start()

In [243]:
await atdome.cmd_closeShutter.start()

RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 21 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 20 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [47]:
await atpne.cmd_openM1Cover.start()

RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 13 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 13 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 13 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 13 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 13 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
fallin

In [237]:
await atpne.cmd_closeM1Cover.start()

RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 21 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 20 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 20 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 20 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 20 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 20 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 20 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 20 messages
RemoteTelemetry(ATPneumatics, 0

In [ ]:
err = await atdome.evt_errorCode.next(flush=False)

In [ ]:
print(err.errorReport)

In [ ]:
await asyncio.sleep(300)

In [ ]:
await asyncio.sleep(300)

In [ ]:
attcs = ATTCS(atmcs=atmcs, 
              atptg=atptg, 
              ataos=ataos, 
              atpneumatics=atpne, 
              athexapod=athex, 
              atdome=atdome, 
              atdometrajectory=atdomtraj, 
              check={"atpneumatics": False, "athexapod": False, "atdome": True, "atdometrajectory": True})

# Slew and Track an Alt/Az position

While the pointing component does not support the slew and track of an alt/az position, the following cells will provide a quick and easy way to perform this task. The user specified a position in Alt/Az and it will use astropy coordinate library to convert it to RA/Dec. 

For that we will need the location of the observatory and the time. Location is defined as an astropy `EarthLocation` and time is taken from the pointing component and then creating an astropy `Time` object.

Obervatory location (lon, lat and height extracted from: https://github.com/lsst/sims_utils)

In [18]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [238]:
alt = 80. * u.deg
az = 0. * u.deg

The next cell will convert the specified Alt/Az into RA/Dec coordinates in ICRS. This coordinate will be used to slew and track. 

In [ ]:
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_atptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

print("slew...")
await atptg.cmd_raDecTarget.set_start(
    targetName="local",
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt.value,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)


# await attcs.slew(ra=cmd_radec.ra.hour, 
#                  dec=cmd_radec.dec.deg, 
#                  rotPA=180.-cmd_elaz.alt.deg,
#                  rot_frame=ATPtg.RotFrame.FIXED,
#                  rot_mode=ATPtg.RotMode.FIELD)

print(f"raDecTarget ra={cmd_radec.ra!r} hour; "
      f"declination={cmd_radec.dec!r} deg")
# script.ataos.cmd_enableCorrection.set(hexapod=True)
# await script.ataos.cmd_enableCorrection.start(timeout=10)



In [ ]:
await atmcs.cmd_startTracking.start()

# Slew and track a RA/Dec target

The next cell shows an example of how to slew and track an RA/Dec target. 

The one caveat with slewing to RA/Dec is getting a proper value for rotPA. We currently want to keep the rotator around zero degrees. We need to improve the handling of this rotation angle on the slew method but right now we compute the angle on the cell. 

In [ ]:
ra = Angle('18:29:01', unit=u.hour)
dec = Angle('-22:26:41', unit=u.deg)
target_name="MOON"

radec = ICRS(ra, dec)

# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=ra.hour,
    declination=dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
# await attcs.slew(ra=ra, 
#                  dec=dec,
#                  rotPA=180.-alt_az.alt.deg,
#                  rot_frame=ATPtg.RotFrame.FIXED,
#                  rot_mode=ATPtg.RotMode.FIELD)

In [ ]:
await atptg.cmd_offsetRADec.set_start(type=1, 
                                      off1=360.,
                                      off2=0.)

In [ ]:
ra = Angle('19:05:24', unit=u.hour)
dec = Angle('+13:51:47', unit=u.deg)
target_name="HR 7235"

radec = ICRS(ra, dec)

# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

await attcs.slew(ra=ra, 
                 dec=dec,
                 rotPA=180.-alt_az.alt.deg,
                 rot_frame=ATPtg.RotFrame.FIXED,
                 rot_mode=ATPtg.RotMode.FIELD)

# Slew and track the MOON (and other solar system body)

In [ ]:
await attcs.slew_to_planet(planet=ATPtg.Planets.MOON)

In [ ]:
await atmcs.cmd_stopTracking.start()

# GRID in RA/Dec

In [ ]:
dra_grid = np.arange(-1., 1.1,0.1)*24./360.
ddec_grid = np.arange(-1., 1.1,0.1)

In [ ]:
grid_ra = np.zeros(len(dra_grid)*len(ddec_grid))
grid_dec = np.zeros(len(dra_grid)*len(ddec_grid))

for i in range(len(ddec_grid)):
    grid_ra[i*len(dra_grid):(i+1)*len(dra_grid)] += dra_grid[::(-1)**i]
    grid_dec[i*len(dra_grid):(i+1)*len(dra_grid)] += ddec_grid[i]

In [ ]:
plt.plot(grid_ra[0], grid_dec[0], 'bo')
plt.plot(grid_ra, grid_dec, ':')
plt.plot(grid_ra[-1], grid_dec[-1], 'go')

In [ ]:
async def wait_center():
    while True:
        opt = input("Center telescope and choose action (press ?<enter>, for list of actions): ")
        
        if opt == 'n':
            print("Next point in the grid...")
            return 1
        elif opt == 'p':
            print("Previous point in the grid...")
            return -1
        elif opt == 't':
            print("Stopping test...")
            return 0
        elif opt == '?':
            print("""Options are:
            n - Go to next point in the grid.
            p - Go to previous point in the grid.
            t - Terminate test.
            """)
        else:
            print("Next point in the grid...")
            return 1

In [ ]:
ra = Angle('19:05:24', unit=u.hour)
dec = Angle('+13:51:47', unit=u.deg)
target_name="HR 7235"

radec = ICRS(ra, dec)

start_from = 0

In [ ]:
ra = Angle('18:29:01', unit=u.hour)
dec = Angle('-22:26:41', unit=u.deg)
target_name="MOON"

radec = ICRS(ra, dec)

In [ ]:
ra = Angle('16:29:24.45970', unit=u.hour)
dec = Angle('-26:25:55.2094', unit=u.deg)
target_name="Alpha Sco"

radec = ICRS(ra, dec)
 


In [ ]:
start_from = 0

In [ ]:
print(f"Grid has {len(grid_ra)} pointings...")
print(f"Starting from {start_from}")

i = start_from
stop_at = 0

# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)


while i < len(grid_ra):
    
    print(f"GRID[{i}]: {grid_ra[i]} x {grid_dec[i]}")

    await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=ra.hour + grid_ra[i],
    declination=dec.deg + grid_dec[i],
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)


#     await attcs.slew(ra=ra.hour + grid_ra[i], 
#                      dec=dec.deg + grid_dec[i],
#                      rotPA=180.-alt_az.alt.deg,
#                      target_name=f"{target_name} RA/Dec GRID[{i}]: {grid_ra[i]} x {grid_dec[i]}",
#                      rot_frame=ATPtg.RotFrame.FIXED,
#                      rot_mode=ATPtg.RotMode.FIELD)
    
    ret_val = await wait_center()
    if ret_val == 0:
        break

    i += ret_val

    if i < 0:
        print(f"Unwrapping index! (i={i})")
        i = 0


# GRID in Alt/Az

In [ ]:
dalt_grid = np.arange(-0.5, +0.6, 0.1)+1.1
daz_grid = np.arange(-0.5, +0.6, 0.1)+0.66

In [ ]:
grid_alt = np.zeros(len(dalt_grid)*len(daz_grid))
grid_az = np.zeros(len(dalt_grid)*len(daz_grid))

for i in range(len(daz_grid)):
    grid_alt[i*len(dalt_grid):(i+1)*len(dalt_grid)] += dalt_grid[::(-1)**i]
    grid_az[i*len(dalt_grid):(i+1)*len(dalt_grid)] += daz_grid[i]

In [ ]:
plt.plot(grid_alt, grid_az, ':')
plt.plot(grid_alt, grid_az, '|')
plt.plot(grid_alt[0], grid_az[0], 'bo')
plt.plot(grid_alt[-1], grid_az[-1], 'go')

In [ ]:
start_from=60

In [ ]:
print(f"Grid has {len(grid_alt)} pointings...")
print(f"Starting from {start_from}")

i = start_from
stop_at = 0

while i < len(grid_alt):
    
    print(f"GRID[{i}]: {grid_alt[i]} x {grid_az[i]}")

    radec = ICRS(ra,
                 dec)

    time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
    curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
    aa = AltAz(location=location, obstime=curr_time_atptg)
    alt_az = radec.transform_to(aa)

    cmd_elaz = AltAz(alt=alt_az.alt+grid_alt[i]*u.deg, az=alt_az.az+grid_az[i]*u.deg, 
                     obstime=curr_time_atptg.tai, 
                     location=location)
    cmd_radec = cmd_elaz.transform_to(ICRS)

    await atptg.cmd_raDecTarget.set_start(
    targetName=f"{target_name} Alt/Az GRID[{i}]: {grid_alt[i]} x {grid_az[i]}",
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)


#     await attcs.slew(ra=cmd_radec.ra.hour,
#                      dec=cmd_radec.dec.deg,
#                      rotPA=180.-alt_az.alt.deg,
#                      target_name=f"{target_name} Alt/Az GRID[{i}]: {grid_alt[i]} x {grid_az[i]}",
#                      rot_frame=ATPtg.RotFrame.FIXED,
#                      rot_mode=ATPtg.RotMode.FIELD)
    
    ret_val = await wait_center()

    if ret_val == 0:
        break
        
    i += ret_val
    
    if i < 0:
        print(f"Unwrapping index! (i={i})")
        i = 0

In [ ]:
radec = ICRS(ra,
             dec)

time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+1.1*u.deg, az=alt_az.az+0.66*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

await atptg.cmd_raDecTarget.set_start(
targetName=f"{target_name} Alt/Az",
targetInstance=ATPtg.TargetInstances.CURRENT,
frame=ATPtg.CoordFrame.ICRS,
epoch=2000,  # should be ignored: no parallax or proper motion
equinox=2000,  # should be ignored for ICRS
ra=cmd_radec.ra.hour,
declination=cmd_radec.dec.deg,
parallax=0,
pmRA=0,
pmDec=0,
rv=0,
dRA=0,
dDec=0,
rotPA=180.-alt_az.alt.deg,
rotFrame=ATPtg.RotFrame.FIXED,
rotMode=ATPtg.RotMode.FIELD,
timeout=10
)

In [ ]:
ra = Angle('16:29:24.45970', unit=u.hour)
dec = Angle('-26:25:55.2094', unit=u.deg)
target_name="Alpha Sco"

radec = ICRS(ra, dec)

In [ ]:
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+0.5*u.deg, az=alt_az.az+1.0*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

await atptg.cmd_raDecTarget.set_start(
targetName=f"{target_name} Alt/Az",
targetInstance=ATPtg.TargetInstances.CURRENT,
frame=ATPtg.CoordFrame.ICRS,
epoch=2000,  # should be ignored: no parallax or proper motion
equinox=2000,  # should be ignored for ICRS
ra=cmd_radec.ra.hour,
declination=cmd_radec.dec.deg,
parallax=0,
pmRA=0,
pmDec=0,
rv=0,
dRA=0,
dDec=0,
rotPA=180.-alt_az.alt.deg,
rotFrame=ATPtg.RotFrame.FIXED,
rotMode=ATPtg.RotMode.FIELD,
timeout=10
)

In [ ]:
await atptg.cmd_stopTracking.start()

In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=ra.hour,
    declination=dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

In [ ]:
az_offset = 0.66*u.deg
el_offset = 1.1*u.deg

In [ ]:
el_offset.to(u.arcsec).value/5, az_offset.to(u.arcsec).value/5

In [ ]:
await atptg.cmd_offsetAzEl.set_start(az=0., 
                                     el=0.)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(az=0., 
                                     el=el_offset.to(u.arcsec).value/5,
                                     num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(az=-az_offset.to(u.arcsec).value/5, 
                                     el=0.,
                                     num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(az=30., 
                                     el=0.,
                                     num=1)

In [ ]:
await atptg.cmd_stopTracking.start()

In [111]:
await atptg.cmd_pointNewFile.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATDome, 1, position) falling behind; read 48 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 10 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [112]:
await atptg.cmd_pointAddData.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [ ]:
await atptg.cmd_pointCloseFile.start()

# Eps Sgr

In [ ]:
ra = Angle("18:24:10.31840", unit=u.hour)
dec = Angle("-34:23:04.6193", unit=u.deg)
target_name="Eps Sgr"

radec = ICRS(ra, dec)

In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(az=0., 
                                     el=-300.,
                                     num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(az=30., 
                                     el=0.,
                                     num=1)

In [ ]:
await atptg.cmd_stopTracking.start()

# Jupiter

In [ ]:
ra = Angle("16:56:34", unit=u.hour)
dec = Angle("-22:18:43", unit=u.deg)
target_name="Jupiter"

radec = ICRS(ra, dec)
 


In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(az=0., 
                                     el=30.,
                                     num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(az=10., 
                                     el=0.,
                                     num=1)

In [ ]:
radec = ICRS(ra,
             dec)

time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+0.62*u.deg, az=alt_az.az+0.62*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

await atptg.cmd_raDecTarget.set_start(
targetName=f"{target_name} Alt/Az",
targetInstance=ATPtg.TargetInstances.CURRENT,
frame=ATPtg.CoordFrame.ICRS,
epoch=2000,  # should be ignored: no parallax or proper motion
equinox=2000,  # should be ignored for ICRS
ra=cmd_radec.ra.hour,
declination=cmd_radec.dec.deg,
parallax=0,
pmRA=0,
pmDec=0,
rv=0,
dRA=0,
dDec=0,
rotPA=180.-alt_az.alt.deg,
rotFrame=ATPtg.RotFrame.FIXED,
rotMode=ATPtg.RotMode.FIELD,
timeout=10
)

In [ ]:
print(f"Grid has {len(grid_alt)} pointings...")
print(f"Starting from {start_from}")

i = start_from
stop_at = 0

while i < len(grid_alt):
    
    print(f"GRID[{i}]: {grid_alt[i]} x {grid_az[i]}")

    radec = ICRS(ra,
                 dec)

    time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
    curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
    aa = AltAz(location=location, obstime=curr_time_atptg)
    alt_az = radec.transform_to(aa)

    cmd_elaz = AltAz(alt=alt_az.alt+grid_alt[i]*u.deg, az=alt_az.az+grid_az[i]*u.deg, 
                     obstime=curr_time_atptg.tai, 
                     location=location)
    cmd_radec = cmd_elaz.transform_to(ICRS)

    await atptg.cmd_raDecTarget.set_start(
    targetName=f"{target_name} Alt/Az GRID[{i}]: {grid_alt[i]} x {grid_az[i]}",
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)


#     await attcs.slew(ra=cmd_radec.ra.hour,
#                      dec=cmd_radec.dec.deg,
#                      rotPA=180.-alt_az.alt.deg,
#                      target_name=f"{target_name} Alt/Az GRID[{i}]: {grid_alt[i]} x {grid_az[i]}",
#                      rot_frame=ATPtg.RotFrame.FIXED,
#                      rot_mode=ATPtg.RotMode.FIELD)
    
    ret_val = await wait_center()

    if ret_val == 0:
        break
        
    i += ret_val
    
    if i < 0:
        print(f"Unwrapping index! (i={i})")
        i = 0

In [ ]:
while True:
    await asyncio.sleep(60)

# Alpha Gru

In [30]:
ra = Angle("22:08:13.98473", unit=u.hour)
dec = Angle("-46:57:39.5078", unit=u.deg)
target_name="Jupiter"

radec = ICRS(ra, dec)

In [62]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 10 messages


58734.210193988765
slew...


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [34]:
radec = ICRS(ra,
             dec)

time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+0.62*u.deg, az=alt_az.az+0.62*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

await atptg.cmd_raDecTarget.set_start(
targetName=f"{target_name} Alt/Az",
targetInstance=ATPtg.TargetInstances.CURRENT,
frame=ATPtg.CoordFrame.ICRS,
epoch=2000,  # should be ignored: no parallax or proper motion
equinox=2000,  # should be ignored for ICRS
ra=cmd_radec.ra.hour,
declination=cmd_radec.dec.deg,
parallax=0,
pmRA=0,
pmDec=0,
rv=0,
dRA=0,
dDec=0,
rotPA=180.-alt_az.alt.deg,
rotFrame=ATPtg.RotFrame.FIXED,
rotMode=ATPtg.RotMode.FIELD,
timeout=10
)

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 58 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 58 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 58 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 59 messages
falling behind; queue contains 31 elements
falling behind; queue 

In [220]:
await atptg.cmd_offsetAzEl.set_start(az=0., 
                                     el=1000.,
                                     num=1)

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 30 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 30 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 29 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 30 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 30 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 30 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 30 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 30 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 30 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 30 messages
RemoteEvent(ATPneumatics, 0, heartbeat) fa

In [221]:
await atptg.cmd_offsetAzEl.set_start(az=0., 
                                     el=1000.,
                                     num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [222]:
await atptg.cmd_offsetAzEl.set_start(az=-500., 
                                     el=0.,
                                     num=1)

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 46 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 12 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [223]:
await atptg.cmd_offsetAzEl.set_start(az=-250., 
                                     el=0.,
                                     num=1)

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 78 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [224]:
await atptg.cmd_offsetAzEl.set_start(az=0., 
                                     el=60.,
                                     num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 33 messages


In [225]:
await atptg.cmd_offsetAzEl.set_start(az=0., 
                                     el=120.,
                                     num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [226]:
await atptg.cmd_offsetAzEl.set_start(az=-60., 
                                     el=0.,
                                     num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 12 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 12 messages


In [227]:
await atptg.cmd_offsetAzEl.set_start(az=0., 
                                     el=30.,
                                     num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 63 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 16 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 16 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 16 messages


In [228]:
await atptg.cmd_offsetAzEl.set_start(az=0., 
                                     el=30.,
                                     num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 20 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [229]:
await atptg.cmd_offsetAzEl.set_start(az=-30., 
                                     el=0.,
                                     num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [125]:
await atptg.cmd_stopTracking.start()

RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 10 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 10 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 49 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 10 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 10 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


# Alf PsA

In [133]:
ra = Angle("22:57:39.04625", unit=u.hour)
dec = Angle("-29:37:20.0533", unit=u.deg)
target_name="Jupiter"

radec = ICRS(ra, dec)
 


In [219]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

RemoteTelemetry(ATDome, 1, position) falling behind; read 18 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 18 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 18 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 18 messages


58734.28169993333
slew...


In [231]:
await atptg.cmd_offsetAzEl.set_start(az=-60., 
                                     el=0.,
                                    num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [244]:
await atptg.cmd_stopTracking.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 33 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [218]:
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATDome, 1, position) falling behind; read 23 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [211]:
await atptg.cmd_pointNewFile.start()
await atptg.cmd_pointAddData.start()

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 92 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 92 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 18 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [190]:
await atmcs.cmd_standby.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 95 messages
RemoteEvent(ATPtg, 0, weatherDataApplied) falling behind; read 47 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elemen

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1487062204, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='ERROR: Command standby rejected while in TrackingDisabledState state.')

In [191]:
await atptg.cmd_offsetAbsorb.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 73 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 72 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 72 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 73 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 73 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 73 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 72 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 73 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 73 messages
RemoteEvent(ATPneumatics, 0, heartbeat) fal

In [ ]:
print("hello")